In [1]:
import pandas as pd


In [9]:
df = pd.read_csv("VentFlowGasMigrationReport.csv")

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39638 entries, 0 to 39637
Data columns (total 16 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   01.Report Date                        39638 non-null  object 
 1   02.Licence Number                     39638 non-null  object 
 2   03.Current Licence Status             39638 non-null  object 
 3   04.Surface Location                   39638 non-null  object 
 4   05.Bottom Location                    39627 non-null  object 
 5   06.Original BA Id                     39638 non-null  object 
 6   07.Licensee at Reporting              39638 non-null  object 
 7   08.Type                               39635 non-null  object 
 8   09.Classification                     39635 non-null  object 
 9   10.Flow Substance                     38666 non-null  object 
 10  11.Flow Rate (m3/day)                 22767 non-null  float64
 11  12.Stabilized S

In [21]:
import pandas as pd
import numpy as np
import re

def clean_ventflow_types(df: pd.DataFrame) -> pd.DataFrame:
    # --- rename to snake_case
    rename_map = {
        '01.Report Date':'report_date',
        '02.Licence Number':'licence',
        '03.Current Licence Status':'licence_status',
        '04.Surface Location':'surface_location',
        '05.Bottom Location':'bottom_location',
        '06.Original BA Id':'original_ba_id',
        '07.Licensee at Reporting':'licensee_at_reporting',
        '08.Type':'event_type',
        '09.Classification':'classification',
        '10.Flow Substance':'flow_substance',
        '11.Flow Rate (m3/day)':'flow_rate_m3d',
        '12.Stabilized Shut In Pressure (kPa)':'sip_kpa',
        '13.Source Depth (mkb)':'source_depth_mkb',
        '14.Ground Water Base (mkb)':'gw_base_mkb',
        '15.Resolution Date':'resolution_date',
        '16.Reported Resolution':'reported_resolution'
    }
    df = df.rename(columns=rename_map).copy()

    # --- strip whitespace globally
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # --- normalize licence (keep raw too)
    df["licence_raw"] = df["licence"]
    df["licence"] = (df["licence"]
        .str.replace(r'^[Ww]\s*', '', regex=True)  # drop leading W
        .str.replace(r'\s+', '', regex=True))      # drop spaces

    # --- parse dates
    for c in ["report_date", "resolution_date"]:
        df[c] = pd.to_datetime(df[c], errors="coerce")

    # --- numeric coercion
    num_cols = ["flow_rate_m3d", "sip_kpa", "source_depth_mkb", "gw_base_mkb"]
    for c in num_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # --- count zeros in depth fields
    zero_source = (df["source_depth_mkb"] == 0).sum()
    zero_gw = (df["gw_base_mkb"] == 0).sum()
    print(f"Zeros in source_depth_mkb: {zero_source}")
    print(f"Zeros in gw_base_mkb: {zero_gw}")

    # --- normalize classification text and cast categoricals
    df["classification"] = (df["classification"]
        .str.strip()
        .str.replace(r'(?i)^ser.*','Serious', regex=True)
        .str.replace(r'(?i)^non[-\s]?ser.*','Non Serious', regex=True))

    cat_cols = [
        "licence_status","event_type","classification","flow_substance",
        "reported_resolution","licensee_at_reporting","surface_location","bottom_location"
    ]
    for c in cat_cols:
        if c in df.columns:
            df[c] = df[c].astype("category")

    # --- derived fields
    df["delta_gw"] = df["gw_base_mkb"] - df["source_depth_mkb"]    # +ve => source deeper than GW base
    df["duration_days"] = (df["resolution_date"] - df["report_date"]).dt.days
    df["is_serious"] = (df["classification"] == "Serious").astype("Int8")
    df["report_year"] = df["report_date"].dt.year.astype("Int16")

    return df

# Usage:
# df = pd.read_csv("VentFlowGasMigrationReport.csv")
# df_clean = clean_ventflow_types(df)


In [22]:
df = pd.read_csv("VentFlowGasMigrationReport.csv")   # your original (as you showed)
df_clean = clean_ventflow_types(df)
df_clean.info(memory_usage="deep")
df_clean.head()

C:\Users\Alireza\AppData\Local\Temp\ipykernel_50400\3096156775.py:28: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\Alireza\AppData\Local\Temp\ipykernel_50400\3096156775.py:38: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[c] = pd.to_datetime(df[c], errors="coerce")
C:\Users\Alireza\AppData\Local\Temp\ipykernel_50400\3096156775.py:38: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[c] = pd.to_datetime(df[c], errors="coerce")


Zeros in source_depth_mkb: 9046
Zeros in gw_base_mkb: 8630
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39638 entries, 0 to 39637
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   report_date            39638 non-null  datetime64[ns]
 1   licence                39638 non-null  object        
 2   licence_status         39638 non-null  category      
 3   surface_location       39638 non-null  category      
 4   bottom_location        39627 non-null  category      
 5   original_ba_id         39638 non-null  object        
 6   licensee_at_reporting  39638 non-null  category      
 7   event_type             39635 non-null  category      
 8   classification         39635 non-null  category      
 9   flow_substance         38666 non-null  category      
 10  flow_rate_m3d          22767 non-null  float64       
 11  sip_kpa                36040 non-null  float64       
 12  s

,report_date,licence,licence_status,surface_location,bottom_location,original_ba_id,licensee_at_reporting,event_type,classification,flow_substance,...,sip_kpa,source_depth_mkb,gw_base_mkb,resolution_date,reported_resolution,licence_raw,delta_gw,duration_days,is_serious,report_year
0,1996-01-29,0075880,Cancelled,10-14-062-05W4,NaN,0TC5,"Vintage Petroleum Canada, Inc. (0TC5)",Vent Flow,Serious,Gas,...,0.0,0.0,0.0,1996-01-29,Died Out,W 0075880,0.0,0.0,1,1996
1,1997-10-27,0107747,Cancelled,16-23-050-04W5,NaN,0MY7,Murphy Canada Exploration Company (0MY7),Vent Flow,Non Serious,Not Converted,...,NaN,0.0,0.0,1997-10-27,Problem Repaired,W 0107747,0.0,0.0,0,1997
2,1996-01-25,0123230,Cancelled,07-20-045-07W4,NaN,0BP8,Obsidian Energy Ltd. (0BP8),Vent Flow,Non Serious,Gas,...,NaN,0.0,0.0,1994-12-07,Problem Repaired,W 0123230,0.0,-414.0,0,1996
3,1995-11-21,0123683,Cancelled,01-35-041-13W4,NaN,0R36,Keles Production Company Ltd. (0R36),Vent Flow,Non Serious,Gas,...,60.0,0.0,0.0,1993-02-10,Problem Repaired,W 0123683,0.0,-1014.0,0,1995
4,1996-01-25,0134794,Cancelled,15-22-040-13W4,NaN,0R46,Husky Oil Operations Limited (0R46),Vent Flow,Non Serious,Gas,...,0.0,0.0,0.0,1996-01-25,Died Out,W 0134794,0.0,0.0,0,1996


In [18]:
df[100:130]

,01.Report Date,02.Licence Number,03.Current Licence Status,04.Surface Location,05.Bottom Location,06.Original BA Id,07.Licensee at Reporting,08.Type,09.Classification,10.Flow Substance,11.Flow Rate (m3/day),12.Stabilized Shut In Pressure (kPa),13.Source Depth (mkb),14.Ground Water Base (mkb),15.Resolution Date,16.Reported Resolution
100,1998-04-17 12:00:00 a.m.,W 0142197,Abandoned,01-03-038-24W4,00/01-03-038-24W4/0,0HE9,Canadian Natural Resources Limited (0HE9),Vent Flow,Serious,Gas,983.00,460.0,600.0,0.00,1996-08-28 12:00:00 a.m.,Problem Repaired
101,2014-10-24 12:00:00 a.m.,W 0445591,Issued,07-10-040-13W4,00/01-03-040-13W4/0,A2J6,Veren Inc. (A2J6),Vent Flow,Non Serious,Gas,0.36,319.0,NaN,291.25,2015-06-02 12:00:00 a.m.,Died Out
102,2020-10-28 12:00:00 a.m.,W 0445591,Issued,07-10-040-13W4,00/01-03-040-13W4/0,A7BT,Karve Energy Inc. (A7BT),Vent Flow,Non Serious,Gas,0.14,3.0,NaN,404.87,NaN,NaN
103,2014-09-12 12:00:00 a.m.,W 0445591,Issued,07-10-040-13W4,00/01-03-040-13W4/0,A2J6,Veren Inc. (A2J6),Vent Flow,Serious,Gas,NaN,319.0,NaN,291.25,NaN,Entry Error-Ignore
104,2007-01-15 12:00:00 a.m.,W 0362553,Abandoned,01-03-043-02W4,00/01-03-043-02W4/0,0039,Repsol Oil & Gas Canada Inc. (0039),Vent Flow,Considered Non Serious,Gas,NaN,195.0,NaN,190.00,2014-07-09 12:00:00 a.m.,Died Out
105,2023-08-23 12:00:00 a.m.,W 0414819,Suspension,16-03-050-04W5,00/01-03-050-04W5/0,A68R,Highvale Energy Limited (A68R),Vent Flow,Non Serious,Gas,0.28,160.0,NaN,556.28,NaN,NaN
106,2011-08-26 12:00:00 a.m.,W 0422035,Abandoned,01-03-050-06W4,00/01-03-050-06W4/0,0R46,Husky Oil Operations Limited (0R46),Vent Flow/Gas Migration,Serious,Gas,NaN,0.0,295.0,130.00,2011-11-20 12:00:00 a.m.,Repaired - SCVF/GM
107,2000-02-15 12:00:00 a.m.,W 0118245,RecCertified,01-03-050-07W4,00/01-03-050-07W4/0,0NX9,"Samson Canada, Ltd. (0NX9)",Vent Flow,Non Serious,Not Converted,NaN,NaN,0.0,0.00,2000-02-15 12:00:00 a.m.,Problem Repaired
108,2012-09-18 12:00:00 a.m.,W 0338838,Suspension,01-03-051-16W5,00/01-03-051-16W5/0,0XL9,ConocoPhillips Canada Operations ULC (0XL9),Vent Flow,Non Serious,Gas,10.00,2.0,NaN,600.00,NaN,NaN
109,2002-11-07 12:00:00 a.m.,W 0090097,RecCertified,01-03-052-04W4,00/01-03-052-04W4/0,0R46,Husky Oil Operations Limited (0R46),Gas Migration,Non Serious,NaN,0.00,0.0,0.0,0.00,2002-07-06 12:00:00 a.m.,Died Out


In [17]:
df_clean[100:130]

,report_date,licence,licence_status,surface_location,bottom_location,original_ba_id,licensee_at_reporting,event_type,classification,flow_substance,...,sip_kpa,source_depth_mkb,gw_base_mkb,resolution_date,reported_resolution,licence_raw,delta_gw,duration_days,is_serious,report_year
100,1998-04-17,0142197,Abandoned,01-03-038-24W4,00/01-03-038-24W4/0,0HE9,Canadian Natural Resources Limited (0HE9),Vent Flow,Serious,Gas,...,460.0,600.0,NaN,1996-08-28 00:00:00,Problem Repaired,W 0142197,NaN,-597.0,1,1998
101,2014-10-24,0445591,Issued,07-10-040-13W4,00/01-03-040-13W4/0,A2J6,Veren Inc. (A2J6),Vent Flow,Non Serious,Gas,...,319.0,NaN,291.25,2015-06-02 00:00:00,Died Out,W 0445591,NaN,221.0,0,2014
102,2020-10-28,0445591,Issued,07-10-040-13W4,00/01-03-040-13W4/0,A7BT,Karve Energy Inc. (A7BT),Vent Flow,Non Serious,Gas,...,3.0,NaN,404.87,NaT,NaN,W 0445591,NaN,NaN,0,2020
103,2014-09-12,0445591,Issued,07-10-040-13W4,00/01-03-040-13W4/0,A2J6,Veren Inc. (A2J6),Vent Flow,Serious,Gas,...,319.0,NaN,291.25,NaT,Entry Error-Ignore,W 0445591,NaN,NaN,1,2014
104,2007-01-15,0362553,Abandoned,01-03-043-02W4,00/01-03-043-02W4/0,0039,Repsol Oil & Gas Canada Inc. (0039),Vent Flow,Considered Non Serious,Gas,...,195.0,NaN,190.00,2014-07-09 00:00:00,Died Out,W 0362553,NaN,2732.0,0,2007
105,2023-08-23,0414819,Suspension,16-03-050-04W5,00/01-03-050-04W5/0,A68R,Highvale Energy Limited (A68R),Vent Flow,Non Serious,Gas,...,160.0,NaN,556.28,NaT,NaN,W 0414819,NaN,NaN,0,2023
106,2011-08-26,0422035,Abandoned,01-03-050-06W4,00/01-03-050-06W4/0,0R46,Husky Oil Operations Limited (0R46),Vent Flow/Gas Migration,Serious,Gas,...,0.0,295.0,130.00,2011-11-20 00:00:00,Repaired - SCVF/GM,W 0422035,-165.00,86.0,1,2011
107,2000-02-15,0118245,RecCertified,01-03-050-07W4,00/01-03-050-07W4/0,0NX9,"Samson Canada, Ltd. (0NX9)",Vent Flow,Non Serious,Not Converted,...,NaN,NaN,NaN,2000-02-15 00:00:00,Problem Repaired,W 0118245,NaN,0.0,0,2000
108,2012-09-18,0338838,Suspension,01-03-051-16W5,00/01-03-051-16W5/0,0XL9,ConocoPhillips Canada Operations ULC (0XL9),Vent Flow,Non Serious,Gas,...,2.0,NaN,600.00,NaT,NaN,W 0338838,NaN,NaN,0,2012
109,2002-11-07,0090097,RecCertified,01-03-052-04W4,00/01-03-052-04W4/0,0R46,Husky Oil Operations Limited (0R46),Gas Migration,Non Serious,NaN,...,0.0,NaN,NaN,2002-07-06 00:00:00,Died Out,W 0090097,NaN,-124.0,0,2002
